In [1]:
import json
import codecs
import uuid
import ast

In [2]:
rasaStoriesFile = 'converted_files/data/stories.md'
rasaUtterancesFile = 'converted_files/data/nlu.md'
rasaDomainFile = 'converted_files/domain.yml'

flowEntities = 'dialogflow/entities/'

In [3]:
entities = []
entity_set = {}

In [4]:
def get_entities_from_domain(domain):
    
    printer = False
    first_line = True
    entities = []
    
    domain_options = ['slots:\n', 'actions:\n', 'responses:\n', 'intents:\n']
    
    with codecs.open(domain, 'r', 'utf-8') as f_out:
        for line in f_out:
            
            first_line = True
            
            if line == 'entities:\n':
                printer = True
                first_line = False
                
            if line in domain_options:
                printer = False
                
            if printer and first_line:
                line = line.strip()
                line = line.replace(' ', '')
                line = line.replace('-', '')
                entities.append(line)

    return entities[:-1]

entities = get_entities_from_domain(rasaDomainFile)

### Write basic entity file

In [5]:
for entity in entities:
    description = {
      "id": str(uuid.uuid4()),
      "name": entity,
      "isOverridable": True,
      "isEnum": False,
      "isRegexp": False,
      "automatedExpansion": True,
      "allowFuzzyExtraction": True
    }    
    
    with open(flowEntities + entity +'.json', 'w') as outfile:
        json.dump(description, outfile,indent=2)
        
    entity_set[entity] = set()


### Get entity examples

In [10]:
def get_entity_examples_from_stories(stories):
    
    entity_set['type'] = set()
    with codecs.open(stories, 'r', 'utf-8') as f_out:
        for line in f_out:
            for entity in entities:
                if entity in line:

                    myLine = line[line.index(entity):]
                    if ':' in myLine:
                        myLine = myLine[myLine.index(':'):]
                        myLine = myLine[3:]
                        val = myLine[:myLine.index('"')]
                        
                        entity_set[entity] = entity_set[entity] |{val} 
                                    
get_entity_examples_from_stories(rasaStoriesFile)

In [11]:
def get_entity_examples_from_nlu(nlu):

    entity_set['type'] = set()
    with codecs.open(nlu, 'r', 'utf-8') as f_out:
        for line in f_out:
            for entity in entities:
                padded_ent = '('+entity+')'
                if padded_ent in line:

                    myLine = line[:line.index(entity)]
                    val = myLine[myLine.index('['):][1:-2]
                        
                    entity_set[entity] = entity_set[entity] |{val} 
                        
get_entity_examples_from_nlu(rasaUtterancesFile)

In [15]:
for ent_set in entity_set:
    
#     print(ent_set)
    
    outp = []
    
    for e in entity_set[ent_set]:
        vals = {
            "value": e,
            "synonyms": [e]
        }
        
        outp.append(vals)
    
    with open(flowEntities + ent_set +'_entires_en.json', 'w') as outfile:
        json.dump(outp, outfile,indent=2)
        